In this step we create or insert to the existing database. We have a table joining both tables from the trusted zone. We join them based on the district and month of crimes and prices.

From the prices table we use:

<ul>
  <li>Month</li>
  <li>District (previously Geo Name)</li>
  <li>AvgPrice</li>
  <li>AvgPriceDetached</li>
  <li>AvgPriceSemiDetached</li>
  <li>AvgPriceTerraced</li>
  <li>AvgPriceFlatOrMaisonette</li>
</ul>

From the crimes table we use group by month, district (previously LSOA name) and crime type. Using that we created a column with a count, mean latitude and mean longitude for each crime type. So the columns in the new table are the follwing:

<ul>
  <li>x Lon</li>
  <li>x Lat</li>
  <li>x Count</li>
</ul>

Where x takes the values of:

<ul>
  <li>Anti-social beahavour</li>
  <li>Criminal damage and arson</li>
  <li>Violence and sexual offences</li>
  <li>Other crime</li>
  <li>Public order</li>
  <li>Vehicle crime</li>
  <li>Burglary</li>
  <li>Drugs</li>
  <li>Theft</li>
  <li>Possesion of weapons</li>
  <li>Shoplifting and Robbery</li>
</ul>

In [ ]:
import duckdb
import pandas as pd
from datetime import date
from dateutil.relativedelta import relativedelta as delta

# Returns a list of 6 months previous to argument month with format YYYY-MM
# ['2022-04', '2022-03' ...] 
def prevSixMonths(month):
    y = int(month.split("-")[0])
    m = int(month.split("-")[1])
    time = date(y, m, 1)

    res = []

    for i in range(1,7):
        res.append(str(time + delta(months=-i))[:-3])

    return res

In [ ]:
def join():
    # Connect to trusted database
    conTrus = duckdb.connect(database="../data/trusted/crimesPrices.db")

    tables = conTrus.execute("SHOW TABLES").df()

    if len(tables['name']) != 0:

        # Import tables to dataframes
        crimesDF = conTrus.execute("SELECT * FROM crimes").df()
        pricesDF = conTrus.execute("SELECT * FROM prices").df()

        # Group all theft related crimes into one modality
        crimesDF.loc[crimesDF['Crime type'].str.lower().str.contains('theft'), 'Crime type'] = "Theft"

        # Reaname 'LSOA name' column to 'District' for joining tables
        crimesDF = crimesDF.rename(columns={'LSOA name' : 'District'})

        # Group crimes by district, type of crime and month
        group = crimesDF.groupby(['District', 'Crime type', 'Month'])

        # Calculate the count of each group and mean latitude and longitude
        counts = group.size().to_frame(name='Count')
        group = (counts
        .join(group.agg({'Longitude': 'mean'}))
        .join(group.agg({'Latitude':  'mean'}))
        .reset_index()
        )

        # Select columns from prices to be in final table
        finalDF = pricesDF[['Month', 'GeoName', 'AveragePrice', 'AveragePriceDetached',
                            'AveragePriceSemiDetached', 'AveragePriceTerraced',
                            'AveragePriceFlatOrMaisonette']]

        # Rename 'GeoName' to 'District' to join tables
        finalDF = finalDF.rename(columns={'GeoName' : 'District'})

        # Perform the join of the two tables
        # Create a count column for each type of crime committed in the past 6 months to the price month
        # So for each month and each district we have count column of each type of crime.
        # The count corresponds to how many crimes were committed of that type in that district the 6 months 
        # (these months correspond to the crimes table) prior to the month column (month corresponds to the prices tables)
        # Also add the mean latitude and longitude for each type of crime and district in the prior 6 months to the month column
        for month in finalDF['Month'].unique():
            for district in finalDF['District'].unique():
                for crime in crimesDF['Crime type'].unique():
                    count = group[(group['District'] == district) &
                                (group['Crime type'] == crime) &
                                (group['Month'].isin(prevSixMonths(month)))][['Count', 'Longitude', 'Latitude']]
                    
                    finalDF.loc[(finalDF['Month'] == month) & (finalDF['District'] == district), crime + ' Count'] = sum(count['Count'])
                    finalDF.loc[(finalDF['Month'] == month) & (finalDF['District'] == district), crime + ' Lon'] = count['Longitude'].mean()
                    finalDF.loc[(finalDF['Month'] == month) & (finalDF['District'] == district), crime + ' Lat'] = count['Latitude'].mean()


        conExp = duckdb.connect("../data/exploitation/crimesPrices.db")
        tables = conExp.execute("SHOW TABLES").df()
        if(len(tables) == 0):
            conExp.execute("CREATE TABLE crimesPrices AS SELECT * FROM finalDF")
        else:
            conExp.execute("INSERT INTO crimesPrices SELECT * from finalDF")

        conExp.close()

    conTrus.close()